# Prepare data with SageMaker Processing

## Setup environment

In [ ]:
import os
import boto3
import sagemaker
from sagemaker.processing import Processor, ProcessingInput, ProcessingOutput

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role() # we are using the notebook instance role for training in this example
bucket = sagemaker_session.default_bucket() # you can specify a bucket name here

## Build and push container

In [ ]:
image_name = 'data-processing-containers'

In [ ]:
!sh ./docker/build_and_push.sh $image_name

In [ ]:
# Replace by your ECR image ID
container = '<your-container-image-uri>' 

## Launch data processing job

In [ ]:
data_processor = Processor(role=role, 
                           image_uri=container, 
                           instance_count=1, 
                           instance_type='ml.m5.xlarge',
                           volume_size_in_gb=30, 
                           max_runtime_in_seconds=1200,
                           base_job_name='data-processing')

In [ ]:
input_folder = '/opt/ml/processing/input'
output_folder = '/opt/ml/processing/output'

data_processor.run(
    arguments= [
        f'--input={input_folder}',
        f'--output={output_folder}'
    ],
    inputs = [
        ProcessingInput(
            input_name='input',
            source='s3://aws-mlops-workshop/reviews/workshop_data/reviews.csv',
            destination=input_folder
        )
    ],
    outputs= [
        ProcessingOutput(
            output_name='preprocessed',
            source=output_folder,
            destination=bucket
        )
    ]
)